[Try this strategy: One stock per week - MarketWatch](urn:publicid:mw_461:281474978891724 "Try this strategy: One stock per week - MarketWatch")

+ Avoid penny stocks.
+ Avoid stocks over 100.
+ All stocks should be highly liquid.
+ We prefer large cap stocks (they move with the market).
+ Avoid stocks reporting earnings in the week ahead.
+ Do not trade stocks expecting other important news (not always possible).
+ Our model portfolio is 50K + margin.




In [ ]:
from __future__ import division

from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import Q1500US,Q500US,default_us_equity_universe_mask,make_us_equity_universe
from quantopian.pipeline.factors import Returns, SimpleMovingAverage, AverageDollarVolume,CustomFactor
from quantopian.pipeline.classifiers.morningstar import Sector
import pandas as pd 

from quantopian.research import run_pipeline

In [ ]:
class CloseOnN(CustomFactor): 
    # Define inputs
    inputs = [USEquityPricing.close]
    window_length = 2 
    def compute(self, today, assets, out, close): 
        out[:] = close[0]


def make_pipeline():
    base_universe = Q500US()

    # 一週間のリターンをとりあえず過去５日間のリターンと考える
    # 本当は，休日の事も考えなくては行けないが，とりあえず決め打ち．
    fiveday_return = Returns(window_length=5) 
    tenday_return = Returns(window_length=10)     
    close = USEquityPricing.close.latest 
    close_5daysbefore = CloseOnN(window_length=5)
    close_10daysbefore = CloseOnN(window_length=10)
    
    # 過去30日の移動平均を取得
    sma30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)
    # 移動平均が10ドル以上
    remove_penny_stocks = sma30 > 10.0
    # 過去30日の売買高
    dollar_volume = AverageDollarVolume(window_length=30)
    # 上位１０％だけを対象
    high_dollar_volume = dollar_volume.percentile_between(90, 100)
    
    pipe = Pipeline(
        screen=base_universe & remove_penny_stocks & high_dollar_volume,
        columns={
            'fiveday_return': fiveday_return,
            'tenday_return': tenday_return,
            'close':close, 
            'close_5daysbefore':close_5daysbefore,
            'close_10daysbefore':close_10daysbefore
        }
    )
    return pipe


In [ ]:
pipe = run_pipeline(make_pipeline(), start_date="2016-1-1", end_date="2018-1-1")

In [ ]:
# ## 5日前のリターンと5日後のリターンの関係を見る
# idx = pd.IndexSlice
# pipe.loc[idx[:,[symbols("AAPL")]],:]

df = pipe.reset_index()
df = df.set_index("level_0").drop("level_0")
df["hoge"] = df.fiveday_return - df.tenday_return

In [ ]:
df[df.index.weekday == 4].plot(x="hoge",y="fiveday_return", kind="scatter", xlim=(-1.0,-0.05))
